<a href="https://colab.research.google.com/github/MAHMOUDPD/modi/blob/master/Solving_a_quadratic_programming_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In this notebook, we will solve a Quadratic Program (QP) using a third-party, non-standard package called `qpsolvers`.

You need to guide yourselves through with an example from https://scaron.info/doc/qpsolvers/quadratic-programming.html .

Let us first install `qpsolvers` into `GoogleColab`.

In [ ]:
%pip install qpsolvers # installing the relavent package

     |████████████████████████████████| 121 kB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for quadprog: filename=quadprog-0.1.11-cp37-cp37m-linux_x86_64.whl size=290758 sha256=7ae9ce016981503885169f40ac1dc1c6639cfb91226cd618fec85a3ca5588ffe
  Stored in directory: /root/.cache/pip/wheels/4a/4e/d7/41034ea11aeef1266df3cae546116cb6094e955c41ae3e2589
Successfully built quadprog


In [ ]:
# useful libraries are imported
import numpy as np
import qpsolvers as qp

Let us modify the Linear Program (LP) from our class into a QP.

**Situation:** \\
Suppose that a company is producing a good within a day.
This company has 6 production sites whose production cost is linear and transportation cost is quadratic. Let $c = (10,17,13,9,15,18)^{\top}$ and $q = (7,5,4,8,2,4)^{\top}$ be the cost and transport coefficients for each site, respectively. This means if Site#1 produces $x_{1}$ items, then its production cost is $c_{1}x_{1} = 10x_{1}$ and its transportation cost is $q_{1}x_{1}^{2} = 7x_{1}^{2}$, and similarly for other sites.
The sites 1-3 are in the same region so that they share the same resources, which limits the total daily production between them to $500,000$ units.
Similarly for the sites 4-6. Their total daily production cannot exceeds $400,000$ units due to the shared resources.
Moreover, due to the limited men and machines, each site can only afford to produce at most $200,000$ units in a day.

At the same time, the company needs to satisfy the demands of $750,000$ on the market while wanting to have the market clearing.

**QUESTION**

Use `qpsolvers` to find an optimal way for this company to produce.

#Solution
The problem can be formulated as quadratic programming as follows:
$$
\begin{array}{ll}
\text{minimize} & f(x) = x^{T}Qx + c^{\top}x = q_{1}x_{1}^{2}+c_{1}x_{1} + \cdots +q_{6}x_{6}^{2} + c_{6}x_{6} \\
\text{subject to}
& x_{1} + x_{2} + x_{3} \leq 500000 \\
& x_{4} + x_{5} + x_{6} \leq 400000 \\
& x_{1} + \cdots + x_{6} = 750000 \\
& 0 \leq x_{i} \leq 200000, \quad \forall i = 1,\cdots,6.
\end{array}
$$
where, $Q$ is diagonal matrix  defined as follows \begin{bmatrix} q_{1}& 0 & 0 & 0 & 0 & 0 \\ 0 & q_{2} & 0 & 0 & 0 &0\\ 0 & 0 & q_{3} & 0 & 0 &0\\ \cdots \\ 0 & 0 & 0 & 0 & 0 &q_{6}  \end{bmatrix}
In the vector-matrix form, this is equivalent to
$$
\begin{array}{ll}
\text{minimize} & f(x) = x^{T}Qx + c^{\top}x \\
\text{subject to}\\
&\begin{bmatrix} 1 & 1 & 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 & 1 &1  \end{bmatrix} \begin{bmatrix} x_{1} \\ \vdots \\ x_{6} \end{bmatrix} \leq \begin{bmatrix} 500000 \\ 400000 \end{bmatrix} \\
&\begin{bmatrix} 1 & 1 & 1 & 1 & 1 &1 \end{bmatrix} \begin{bmatrix} x_{1} \\ \vdots \\ x_{6} \end{bmatrix}  = 750000 \\
&0 \leq x_{i} \leq 200000, \quad \forall i = 1,\cdots,6.
\end{array}
$$

In [ ]:
# re-writing the above QP problem in to the form that the solver can take as input
q1 = np.array([7.,5.,4.,8.,2.,4.])
P = np.diag(q1) # creating a diagonal matrix 
q = np.array([10., 17., 13., 9., 15., 18.])
G = np.array([[1., 1., 1., 0., 0., 0.], [0., 0., 0., 1., 1., 1.]])
h = np.array([500000., 400000.])
A = np.array([[1., 1., 1., 1., 1., 1.]])
b = np.array([750000.])
lb = np.array([0.,0.,0.,0.,0.,0])
ub = np.array([200000.,200000., 200000.,200000.,200000.,200000. ])

x = qp.solve_qp(P, q, G, h, A, b, lb, ub)
print(f"The optimal items that should be produce by the six Sites is:\n {x}")

The optimal items that should be produce by the six Sites is:
 [ 84337.86746988 118071.61445783 147590.51807229  66667.41666667
 200000.         133332.58333333]


In [ ]:
sum(x) # x = [ 84337.86746988 118071.61445783 147590.51807229  66667.41666667 200000. 133332.58333333]

750000.0000000001